In [1]:
include("../src/SpinonStructure.jl")
include("../src/BZMath.jl")


using .BZmath
using .SpinonStructure
using StaticArrays
using SparseArrays
using LinearAlgebra
using BenchmarkTools


In [2]:
sim = SimulationParameters("0flux_xxz_test",
    lattice=geom.PyroPrimitive(2,2,3),
    A=zeros(2*2*3,4),
    Jpm=-0.046,
    B=[0.,0.,0.],
    n_samples=10000
    )

# sim = SimulationParameters("0flux_xxz_test",
#     lattice=geom.PyroPrimitive(1,1,1),
#     A=[0 0 0 0],
#     Jpm=-0.046,
#     B=[0.,0.,0.],
#     n_samples=10000
#     )


csim = CompiledModel(sim)



CompiledModel(SimulationParameters([0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], -0.046, [0.0, 0.0, 0.0], Main.SpinonStructure.PyrochloreGeometry.PyroPrimitive([2, 2, 3], SVector{3, Int64}[[0, 0, 0], [4, 4, 0], [8, 8, 0], [4, 0, 4], [8, 4, 4], [12, 8, 4], [0, 4, 4], [4, 8, 4], [8, 12, 4], [4, 4, 8]  …  [10, 10, 2], [6, 2, 6], [10, 6, 6], [14, 10, 6], [2, 6, 6], [6, 10, 6], [10, 14, 6], [6, 6, 10], [10, 10, 10], [14, 14, 10]], SVector{3, Int64}[[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1], [5, 5, 1], [5, 3, -1], [3, 5, -1], [3, 3, 1], [9, 9, 1], [9, 7, -1]  …  [3, 5, 7], [3, 3, 9], [9, 9, 9], [9, 7, 7], [7, 9, 7], [7, 7, 9], [13, 13, 9], [13, 11, 7], [11, 13, 7], [11, 11, 9]]), 2.0, 10000, "0flux_xxz_test"), [13, 13, 13, 19, 19, 16, 1, 1, 1, 7  …  10, 10, 11, 11, 11, 11, 12, 12, 12, 12], [19, 16, 15, 16, 15, 15, 7, 4, 2, 4  …  19, 24, 23, 17, 20, 22, 24, 18, 21, 23], ComplexF64[-0.0115 - 0.0im, -0.0115 - 0.0im, -0.0115 - 0.0im, -0.0115 - 0.0im, -0.0115 - 

In [3]:

# Check correctness of XXZ
for i=1:1000
    K = @SVector rand(3)
    ham1 = SpinonStructure.calc_xxz_hopping_fast(sim, K)
    ham2 = SpinonStructure.calc_hopping(csim, K)

    diff = norm( ham1 - Matrix(ham2))
    if diff > 1e-10
        println("Discrepancy found!")
        println(ham1)
        println(ham2)
    end
end




In [4]:

K = @SVector rand(3)
# benchmark
println("benchmarking original form:")
@benchmark SpinonStructure.calc_xxz_hopping_fast(sim, K)


benchmarking original form:


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  38.792 μs …  46.111 ms  ┊ GC (min … max):  0.00% … 99.83%
 Time  (median):     40.917 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   48.818 μs ± 461.735 μs  ┊ GC (mean ± σ):  11.69% ±  3.61%

      ▂█▅▁                                                      
  ▂▄██████▆▃▂▁▁▂▂▂▃▃▂▂▂▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  38.8 μs         Histogram: frequency by time         57.2 μs <

 Memory estimate: 102.86 KiB, allocs estimate: 2691.

In [5]:

println("benchmarking new form:")
@benchmark SpinonStructure.calc_hopping(csim, K)


benchmarking new form:


BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):   6.600 μs …  13.095 ms  ┊ GC (min … max):  0.00% … 99.86%
 Time  (median):      9.200 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   12.865 μs ± 134.942 μs  ┊ GC (mean ± σ):  25.92% ±  6.39%

                    ▁█▇▄▅█▆▂                                    
  ▂▆▅▂▁▁▁▁▁▁▁▁▁▂▂▃▃▅████████▆▇▆▆▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  6.6 μs          Histogram: frequency by time         13.3 μs <

 Memory estimate: 43.95 KiB, allocs estimate: 38.

In [6]:
sim_Btest = SimulationParameters("0flux_B_test",
    lattice=geom.PyroPrimitive(2,5,5),
    A=zeros(2*5*5,4),
    Jpm=0,
    B=[0.3,0.,0.],
    n_samples=10000
    )

csim_Btest = CompiledModel(sim_Btest)


CompiledModel(SimulationParameters([0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], 0.0, [0.3, 0.0, 0.0], Main.SpinonStructure.PyrochloreGeometry.PyroPrimitive([2, 5, 5], SVector{3, Int64}[[0, 0, 0], [4, 4, 0], [8, 8, 0], [12, 12, 0], [16, 16, 0], [4, 0, 4], [8, 4, 4], [12, 8, 4], [16, 12, 4], [20, 16, 4]  …  [14, 6, 18], [18, 10, 18], [22, 14, 18], [26, 18, 18], [30, 22, 18], [18, 6, 22], [22, 10, 22], [26, 14, 22], [30, 18, 22], [34, 22, 22]], SVector{3, Int64}[[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1], [5, 5, 1], [5, 3, -1], [3, 5, -1], [3, 3, 1], [9, 9, 1], [9, 7, -1]  …  [23, 13, 19], [23, 11, 21], [29, 17, 21], [29, 15, 19], [27, 17, 19], [27, 15, 21], [33, 21, 21], [33, 19, 19], [31, 21, 19], [31, 19, 21]]), 2.0, 10000, "0flux_B_test"), [51, 51, 51, 76, 76, 71, 1, 1, 1, 26  …  48, 48, 49, 49, 49, 49, 50, 50, 50, 50], [76, 71, 55, 71, 55, 55, 26, 6, 2, 6  …  93, 97, 99, 74, 94, 98, 100, 75, 95, 99], ComplexF64[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im,

In [7]:
nerror = 0
# Check correctness of Sz
for i=1:10000
    K = @SVector rand(3)
    ham1 = SpinonStructure.calc_nn_hopping(sim_Btest, K)
    ham2 = SpinonStructure.calc_hopping(csim_Btest, K)

    diff = norm( ham1 - Matrix(ham2))
    if diff > 1e-10
        println("Discrepancy found!")
        println(ham1)
        println(ham2)
        nerror += 1
    end
end
if nerror == 0
    println("Test passed")
end

Test passed


In [8]:
@benchmark SpinonStructure.calc_nn_hopping(sim_Btest, K)


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  56.167 μs …   3.177 ms  ┊ GC (min … max):  0.00% … 96.66%
 Time  (median):     67.708 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   76.997 μs ± 113.647 μs  ┊ GC (mean ± σ):  10.48% ±  7.12%

                ▂██▇█▇▂                                         
  ▂▄▄▄▃▂▂▁▁▁▁▁▂▅████████▆▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  56.2 μs         Histogram: frequency by time         94.7 μs <

 Memory estimate: 270.39 KiB, allocs estimate: 3806.

In [9]:
@benchmark SpinonStructure.calc_hopping(csim_Btest, K)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  22.459 μs …  11.445 ms  ┊ GC (min … max):  0.00% … 99.52%
 Time  (median):     32.667 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   41.932 μs ± 168.538 μs  ┊ GC (mean ± σ):  20.29% ±  5.94%

                   ▇█▇▄▄▄▄▃                                     
  ▂▂▁▂▁▁▁▁▁▁▁▁▂▃▄▆▇█████████▇▅▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  22.5 μs         Histogram: frequency by time         51.1 μs <

 Memory estimate: 172.20 KiB, allocs estimate: 38.